Importando as bibliotecas necessárias


In [2]:
import pandas as pd
import requests
import pdb
from plyer import notification
from datetime import datetime

Conectando com a API

In [3]:
url = "https://restcountries.com/v3.1/all"
response = requests.get(url)

Criando uma função de alerta para o status da conexão

In [4]:
def alerta(codigo, base, etapa):
    now = datetime.now()
    formatted_now = now.strftime("%d/%m/%Y %H:%M:%S")

    msg = f"Status do carregamento da base {base} na etapa {etapa}.\n{formatted_now}"

    if response.status_code < 200 or response.status_code > 299:
        title = f'Erro na conexão com a API. Código status: {response.status_code}'
    else:
        title = f'Conexão com a API bem-sucedida. Código status: {response.status_code}'
        dados = response.json()
        df = pd.DataFrame(dados)

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

alerta(codigo = response.status_code,
        base = url,
        etapa = "Importação API")

Formando um Dataframe

In [5]:
if  response.status_code >= 200 and response.status_code <= 299:
    dados = response.json()
    df = pd.DataFrame(dados)

df_formatado = pd.DataFrame(dados)
df_formatado['Nome'] = df_formatado['name'].apply(lambda x: x['common'])
df_formatado['Mapa'] = df_formatado['maps'].apply(lambda x: x['googleMaps'])
df_formatado['Região'] = df_formatado['region']

df_formatado['Nome'] = df_formatado['Nome'].astype(str)
df_formatado['Região'] = df_formatado['Região'].astype(str)
df_formatado['Mapa'] = df_formatado['Mapa'].astype(str)

df_formatado = df_formatado[['Nome', 'Região', 'Mapa']]

df_formatado.head()


,Nome,Região,Mapa
0,Andorra,Europe,https://goo.gl/maps/JqAnacWE2qEznKgw7
1,French Southern and Antarctic Lands,Antarctic,https://goo.gl/maps/6ua6CX1m4w1xF2Em7
2,Laos,Asia,https://goo.gl/maps/F3asVB7sRKgSnwbE7
3,Canada,Americas,https://goo.gl/maps/jmEVLugreeqiZXxbA
4,Nigeria,Africa,https://goo.gl/maps/LTn417qWwBPFszuV9


Colocando o resultado em um CSV


In [6]:
df_formatado.to_csv('restcountries.csv', index=False)